# Splitting the data

In [ ]:
!pip install split-folders

In [4]:
import splitfolders
input_folder = 'D:/Grad_Proj/colon_histo/main/New/'
output_folder = 'D:/Grad_Proj/colon_histo/main/New/after_split'

In [5]:
# splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.7, 0.2, 0.1), group_prefix=None)

Copying files: 26376 files [02:27, 178.35 files/s]


# Training the model

In [21]:
def load_image2(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255
    return img_tensor

In [29]:
path = 'D:/Grad_Proj/endo_class/endo_model_class.h5'
custom_objects = {'KerasLayer': hub.KerasLayer}

histo_model = tf.keras.models.load_model(path, custom_objects=custom_objects)


In [28]:
# img = Image.open('D:/image_processing/Lung/before_split/Benign/lungn1.jpeg')
# img = Image.open('D:/image_processing/Breast_Cancer/augmented_data/after_split/val/Grade_3/Grade_3 (17).JPG')

# endo
# ''
img = load_image2('D:/Grad_Proj/NCT-CRC-HE-100K/LYM/LYM-AACHDAGQ.tif')
prediction = histo_model.predict(img)
classes = np.argmax(prediction, axis=1)[0]
if classes == 0:
    print("Adenocarcinoma (Cancerous)")
elif classes == 1:
    print("Benign (Non Cancerous)")
else:
    print("False data entry")

1/1 [==============================] - 0s 35ms/step
Benign (Non Cancerous)


In [20]:
import os
import numpy as np 
import pandas as pd
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
from pathlib import Path
import tensorflow_hub as hub    
from tensorflow import keras
# import cv2
import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
%matplotlib inline

In [7]:
batch_size = 16
IMG_SHAPE = 299
base_dir = 'C:/Users/dell g3/Downloads/colon_histo/sub'
val_base_dir = 'C:/Users/dell g3/Downloads/colon_histo/sub'


In [8]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(val_base_dir, 'val')
test_dir = os.path.join(val_base_dir, 'test')


In [4]:
training_data = tf.keras.utils.image_dataset_from_directory(directory = train_dir,
                                                           labels = 'inferred',
                                                           label_mode='binary',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           shuffle=True,
                                                           color_mode='rgb')


Found 8000 files belonging to 2 classes.


In [5]:
val_data = tf.keras.utils.image_dataset_from_directory(directory = val_dir,
                                                           labels = 'inferred',
                                                           label_mode='binary',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 1000 files belonging to 2 classes.


In [5]:
test_data = tf.keras.utils.image_dataset_from_directory(directory = test_dir,
                                                           labels = 'inferred',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 1000 files belonging to 2 classes.


In [7]:
@tf.function
def map_fn(img, label):
    image_height = IMG_SHAPE
    image_width = IMG_SHAPE
    img = tf.image.resize(img, [image_height, image_width])
    img /= 255
    return img, label
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

In [8]:
triann = training_data.map(map_fn)
vall = val_data.map(map_fn)
testt = test_data.map(map_fn)

In [9]:
IMAGE_RES = 299

URL = "https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)
with tf.device(device_name=device):
    
    model_inceptionres = tf.keras.Sequential([
      feature_extractor,
        tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(2)
    ])

    model_inceptionres.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              54336736  
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 55,124,706
Trainable params: 787,970
Non-trainable params: 54,336,736
_________________________________________________________________


In [10]:
with tf.device(device_name=device):
    
    model_inceptionres.compile(
      optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001,
      momentum=0.9),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

In [11]:
checkpoint_path = "C:/Users/dell g3/Downloads/colon_histo/sub/cpk/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [12]:
EPOCHS = 10
with tf.device(device_name=device):
    
    history = model_inceptionres.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall,
                        callbacks=[cp_callback])

Epoch 1/10
500/500 [==============================] - ETA: 0s - loss: 0.0888 - accuracy: 0.9734
Epoch 1: saving model to C:/Users/dell g3/Downloads/colon_histo/sub/cpk\cp.ckpt
500/500 [==============================] - 172s 313ms/step - loss: 0.0888 - accuracy: 0.9734 - val_loss: 0.0314 - val_accuracy: 0.9960
Epoch 2/10
500/500 [==============================] - ETA: 0s - loss: 0.0283 - accuracy: 0.9936
Epoch 2: saving model to C:/Users/dell g3/Downloads/colon_histo/sub/cpk\cp.ckpt
500/500 [==============================] - 158s 315ms/step - loss: 0.0283 - accuracy: 0.9936 - val_loss: 0.0195 - val_accuracy: 0.9960
Epoch 3/10
500/500 [==============================] - ETA: 0s - loss: 0.0203 - accuracy: 0.9950
Epoch 3: saving model to C:/Users/dell g3/Downloads/colon_histo/sub/cpk\cp.ckpt
500/500 [==============================] - 160s 319ms/step - loss: 0.0203 - accuracy: 0.9950 - val_loss: 0.0144 - val_accuracy: 0.9980
Epoch 4/10
500/500 [==============================] - ETA: 0s - los

In [13]:
loss, accuracy_d = model_inceptionres.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

63/63 - 17s - loss: 0.0096 - accuracy: 0.9980 - 17s/epoch - 272ms/step
 model, accuracy: 99.80%


In [16]:
model_inceptionres.save('C:/Users/dell g3/Downloads/colon_histo/sub/saved_model/my_model')

INFO:tensorflow:Assets written to: C:/Users/dell g3/Downloads/colon_histo/sub/saved_model/my_model\assets


INFO:tensorflow:Assets written to: C:/Users/dell g3/Downloads/colon_histo/sub/saved_model/my_model\assets


In [17]:
model = tf.keras.models.load_model('C:/Users/dell g3/Downloads/colon_histo/sub/saved_model/my_model')


In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('C:/Users/dell g3/Downloads/colon_histo/sub/saved_model/histo_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DELLG3~1\AppData\Local\Temp\tmp1ehgwgvl\assets


INFO:tensorflow:Assets written to: C:\Users\DELLG3~1\AppData\Local\Temp\tmp1ehgwgvl\assets


## Re-training

In [2]:
batch_size = 16
IMG_SHAPE = 299
base_dir = 'C:/Users/dell g3/Downloads/colon_histo/sub'
val_base_dir = 'C:/Users/dell g3/Downloads/colon_histo/sub'


In [3]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(val_base_dir, 'val')
test_dir = os.path.join(val_base_dir, 'test')


In [4]:
training_data = tf.keras.utils.image_dataset_from_directory(directory = train_dir,
                                                           labels = 'inferred',
                                                           label_mode='binary',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           shuffle=True,
                                                           color_mode='rgb')

Found 11060 files belonging to 2 classes.


In [5]:
val_data = tf.keras.utils.image_dataset_from_directory(directory = val_dir,
                                                           labels = 'inferred',
                                                           label_mode='binary',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 1000 files belonging to 2 classes.


In [6]:
test_data = tf.keras.utils.image_dataset_from_directory(directory = test_dir,
                                                           labels = 'inferred',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 1483 files belonging to 2 classes.


In [7]:
@tf.function
def map_fn(img, label):
    image_height = IMG_SHAPE
    image_width = IMG_SHAPE
    img = tf.image.resize(img, [image_height, image_width])
    img /= 255
    return img, label
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

In [8]:
triann = training_data.map(map_fn)
vall = val_data.map(map_fn)
testt = test_data.map(map_fn)


In [9]:
model = tf.keras.models.load_model('C:/Users/dell g3/Downloads/colon_histo/sub/saved_model/my_model')

In [10]:
checkpoint_path = "C:/Users/dell g3/Downloads/colon_histo/sub/cpk/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [11]:
EPOCHS = 20
with tf.device(device_name=device):
    
    history = model.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall,
                        callbacks=[cp_callback])

Epoch 1/20
692/692 [==============================] - ETA: 0s - loss: 0.0424 - accuracy: 0.9841
Epoch 1: saving model to C:/Users/dell g3/Downloads/colon_histo/sub/cpk\cp.ckpt
692/692 [==============================] - 248s 317ms/step - loss: 0.0424 - accuracy: 0.9841 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 2/20
692/692 [==============================] - ETA: 0s - loss: 0.0272 - accuracy: 0.9911
Epoch 2: saving model to C:/Users/dell g3/Downloads/colon_histo/sub/cpk\cp.ckpt
692/692 [==============================] - 216s 311ms/step - loss: 0.0272 - accuracy: 0.9911 - val_loss: 0.0057 - val_accuracy: 1.0000
Epoch 3/20
692/692 [==============================] - ETA: 0s - loss: 0.0210 - accuracy: 0.9929
Epoch 3: saving model to C:/Users/dell g3/Downloads/colon_histo/sub/cpk\cp.ckpt
692/692 [==============================] - 223s 322ms/step - loss: 0.0210 - accuracy: 0.9929 - val_loss: 0.0058 - val_accuracy: 0.9990
Epoch 4/20
692/692 [==============================] - ETA: 0s - los

In [12]:
loss, accuracy_d = model.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

93/93 - 28s - loss: 0.0883 - accuracy: 0.9730 - 28s/epoch - 298ms/step
 model, accuracy: 97.30%


In [13]:
model.save('C:/Users/dell g3/Downloads/colon_histo/sub/saved_model/my_model_3.h5', save_format='h5')

### Validate yolo annotaions

In [2]:
# import cv2
# import os
# import random


# #全局变量进行路径配置
# label_folder = 'C:/Users/dell g3/Downloads/PolypsSet/train2019/Annotation/yolo_AdenoAnnot2/'  #检测结果存放文件夹labels路径

# raw_images_folder = 'C:/Users/dell g3/Downloads/PolypsSet/train2019/Data_Splitted/adenomatous/'  #检查图片存放文件夹raw_images路径

# save_images_folder = 'C:/Users/dell g3/Downloads/PolypsSet/train2019/save_image2/'  #保存图片文件夹save_image路径

# name_list_path = 'C:/Users/dell g3/Downloads/PolypsSet/train2019/name_list.txt'  #里面有检测图片名称txt文件路径

# classes_path = 'C:/Users/dell g3/Downloads/PolypsSet/train2019/classes.txt'


# def plot_one_box(x, image, color=None, label=None, line_thickness=None):
#     # Plots one bounding box on image img
#     tl = line_thickness or round(0.002 * (image.shape[0] + image.shape[1]) / 2) + 1  # line/font thickness
#     color = color or [random.randint(0, 255) for _ in range(3)]
#     c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
#     cv2.rectangle(image, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
#     if label:
#         tf = max(tl - 1, 1)  # font thickness
#         t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
#         c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
#         cv2.rectangle(image, c1, c2, color, -1, cv2.LINE_AA)  # filled
#         cv2.putText(image, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

# #函数：在一幅图片对应位置上加上矩形框  image_name 图片名称不含后缀 
# def draw_box_on_image(image_name, classes, colors, label_folder, raw_images_folder, save_images_folder ):
#     txt_path  = os.path.join(label_folder,'%s.txt'%(image_name))  #本次检测结果txt路径
#     print(image_name)
#     if image_name == '.DS_Store':
#         return 0
#     image_path = os.path.join( raw_images_folder,'%s.jpg'%(image_name))  #本次原始图片jpg路径
    
#     save_file_path = os.path.join(save_images_folder,'%s.jpg'%(image_name)) #本次保存图片jpg路径
    
#     # flag_people_or_car_data = 0  #变量 代表类别
#     source_file = open(txt_path) if os.path.exists(txt_path) else []
#     image = cv2.imread(image_path)
#     try:
#         height, width, channels = image.shape
#     except:
#         print('no shape info.')
#         return 0

#     box_number = 0
#     for line in source_file: #例遍 txt文件得每一行
#         staff = line.split() #对每行内容 通过以空格为分隔符对字符串进行切片
#         class_idx = int(staff[0])

#         x_center, y_center, w, h = float(staff[1])*width, float(staff[2])*height, float(staff[3])*width, float(staff[4])*height
#         x1 = round(x_center-w/2)
#         y1 = round(y_center-h/2)
#         x2 = round(x_center+w/2)
#         y2 = round(y_center+h/2)     
        
#         # if class_idx == 0: 
#         #     draw_people_tangle = cv2.rectangle(image, (x1,y1),(x2,y2),(0,0,255),2)   # 画框操作  红框  宽度为1
#         #     cv2.imwrite(save_file_path,draw_people_tangle)  #画框 并保存
#         # elif class_idx == 1:
#         #     draw_car_tangle = cv2.rectangle(image,(x1,y1),(x2,y2),(0,255,0),2)     # 画框操作  绿框  宽度为1
#         #     cv2.imwrite(save_file_path,draw_car_tangle)  #画框 并保存

#         plot_one_box([x1,y1,x2,y2], image, color=colors[class_idx], label=classes[class_idx], line_thickness=None)

#         cv2.imwrite(save_file_path,image) 

#         box_number += 1
#     return box_number
    

# #函数：通过保存有原始图片得文件夹，生成写有所有检测图片名称（不带后缀）得txt
# def make_name_list(raw_images_folder, name_list_path):

#     image_file_list = os.listdir(raw_images_folder) #得到该路径下所有文件名称带后缀

#     text_image_name_list_file=open(name_list_path,'w')  #以写入的方式打开txt ，方便更新 不要用追加写

#     for  image_file_name in image_file_list:#例遍写入
#         image_name,file_extend = os.path.splitext(image_file_name)  # 去掉扩展名
#         text_image_name_list_file.write(image_name+'\n') #写入
    
#     text_image_name_list_file.close()


# if __name__ == '__main__':           # 只有在文件作为脚本文件直接执行时才执行下面代码  

#     make_name_list(raw_images_folder, name_list_path) #执行写入txt函数

#     classes = image_names = open(classes_path).read().strip().split('\n')
#     random.seed(42)
#     colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(classes))]

#     image_names = open(name_list_path).read().strip().split() #得到图片名字不带后缀

#     box_total = 0
#     image_total = 0
#     for image_name in image_names: #例遍图片名称
#         box_num = draw_box_on_image(image_name, classes, colors, label_folder, raw_images_folder, save_images_folder)#对图片画框
#         box_total += box_num
#         image_total += 1
#         print('Box number:', box_total, 'Image number:',image_total)

1
Box number: 1 Image number: 1
100
Box number: 2 Image number: 2
10000
Box number: 3 Image number: 3
10002
Box number: 4 Image number: 4
10003
Box number: 5 Image number: 5
10005
Box number: 6 Image number: 6
10006
Box number: 7 Image number: 7
10007
Box number: 8 Image number: 8
10010
Box number: 9 Image number: 9
10012
Box number: 10 Image number: 10
10016
Box number: 11 Image number: 11
10017
Box number: 12 Image number: 12
10018
Box number: 13 Image number: 13
10020
Box number: 14 Image number: 14
10021
Box number: 15 Image number: 15
10023
Box number: 16 Image number: 16
10024
Box number: 17 Image number: 17
10025
Box number: 18 Image number: 18
10026
Box number: 19 Image number: 19
10027
Box number: 20 Image number: 20
10028
Box number: 21 Image number: 21
10029
Box number: 22 Image number: 22
1003
Box number: 23 Image number: 23
10031
Box number: 24 Image number: 24
10032
Box number: 25 Image number: 25
10035
Box number: 26 Image number: 26
10036
Box number: 27 Image number: 27

## Reloading the model


In [7]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions


In [14]:
path = 'C:/Users/dell g3/Downloads/colon_histo/sub/saved_model/my_model_2/my_model_2.h5'
model = tf.keras.models.load_model((path), custom_objects={'KerasLayer':hub.KerasLayer})

KeyboardInterrupt: 

In [ ]:
img_path1 = 'C:/Users/dell g3/Downloads/colon_histo/sub/test/adenocarcinoma/colonca15.jpeg'
img_path2 = 'C:/Users/dell g3/Downloads/colon_histo/sub/test/benign/colonn15.jpeg'

In [ ]:
img1 = image.load_img(img_path1, target_size=(299, 299))
img2 = image.load_img(img_path2, target_size=(299, 299))

img_array1 = image.img_to_array(img1)
img_batch1 = np.expand_dims(img_array1, axis=0)

img_array2 = image.img_to_array(img2)
img_batch2 = np.expand_dims(img_array2, axis=0)

img_preprocessed1 = preprocess_input(img_batch1)
img_preprocessed2 = preprocess_input(img_batch2)


In [ ]:
prediction1 = model.predict(img_preprocessed1)
prediction2 = model.predict(img_preprocessed2)

In [ ]:
print(decode_predictions(prediction1, top=3)[0])

In [ ]:
print(decode_predictions(prediction2, top=3)[0])

### Custom metrics

In [51]:
from keras import backend as K


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras


def specificity(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    return tn / (tn + fp + K.epsilon())


def negative_predictive_value(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))
    return tn / (tn + fn + K.epsilon())


def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))


def fbeta(y_true, y_pred, beta=2):
    y_pred = K.clip(y_pred, 0, 1)

    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    fp = K.sum(K.round(K.clip(y_pred - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    num = (1 + beta ** 2) * (p * r)
    den = (beta ** 2 * p + r + K.epsilon())
    return K.mean(num / den)


def matthews_correlation_coefficient(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())


def equal_error_rate(y_true, y_pred):
    n_imp = tf.count_nonzero(tf.equal(y_true, 0), dtype=tf.float32) + tf.constant(K.epsilon())
    n_gen = tf.count_nonzero(tf.equal(y_true, 1), dtype=tf.float32) + tf.constant(K.epsilon())

    scores_imp = tf.boolean_mask(y_pred, tf.equal(y_true, 0))
    scores_gen = tf.boolean_mask(y_pred, tf.equal(y_true, 1))

    loop_vars = (tf.constant(0.0), tf.constant(1.0), tf.constant(0.0))
    cond = lambda t, fpr, fnr: tf.greater_equal(fpr, fnr)
    body = lambda t, fpr, fnr: (
        t + 0.001,
        tf.divide(tf.count_nonzero(tf.greater_equal(scores_imp, t), dtype=tf.float32), n_imp),
        tf.divide(tf.count_nonzero(tf.less(scores_gen, t), dtype=tf.float32), n_gen)
    )
    t, fpr, fnr = tf.while_loop(cond, body, loop_vars, back_prop=False)
    eer = (fpr + fnr) / 2

    return eer

## Re-Training & Testing Inception_resnet_v2 on 3 classes

In [2]:
batch_size = 16
IMG_SHAPE = 299
base_dir = 'D:/Grad_Proj/colon_histo/main'
val_base_dir = 'D:/Grad_Proj/colon_histo/main'

In [3]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(val_base_dir, 'val')
test_dir = os.path.join(val_base_dir, 'test')

In [4]:
training_data = tf.keras.utils.image_dataset_from_directory(directory = train_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           shuffle=True,
                                                           color_mode='rgb')

Found 48236 files belonging to 3 classes.


In [5]:
val_data = tf.keras.utils.image_dataset_from_directory(directory = val_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 8582 files belonging to 3 classes.


In [10]:
test_data = tf.keras.utils.image_dataset_from_directory(directory = test_dir,
                                                           labels = 'inferred',
                                                           batch_size=1,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 4612 files belonging to 3 classes.


In [13]:
@tf.function
def map_fn(img, label):
    image_height = IMG_SHAPE
    image_width = IMG_SHAPE
    img = tf.image.resize(img, [image_height, image_width])
    img /= 255
    return img, label
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

In [14]:
triann = training_data.map(map_fn)
vall = val_data.map(map_fn)
testt = test_data.map(map_fn)

In [20]:
IMAGE_RES = 299

URL = "https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)
with tf.device(device_name=device):
    
    model_inceptionres = tf.keras.Sequential([
      feature_extractor,
        tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(3)
    ])

    model_inceptionres.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1536)              54336736  
                                                                 
 dense_2 (Dense)             (None, 512)               786944    
                                                                 
 dense_3 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 55,125,219
Trainable params: 788,483
Non-trainable params: 54,336,736
_________________________________________________________________


In [21]:
with tf.device(device_name=device):
    
    model_inceptionres.compile(
      optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001,
      momentum=0.9),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

In [25]:
checkpoint_path = "D:/Grad_Proj/colon_histo/main/cpk/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [26]:
EPOCHS = 1
with tf.device(device_name=device):
    
    history = model_inceptionres.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall,
                        callbacks=[cp_callback])

3015/3015 [==============================] - ETA: 0s - loss: 0.0107 - accuracy: 0.9970
Epoch 1: saving model to D:/Grad_Proj/colon_histo/main/cpk\cp.ckpt
3015/3015 [==============================] - 1020s 338ms/step - loss: 0.0107 - accuracy: 0.9970 - val_loss: 0.0157 - val_accuracy: 0.9962


In [27]:
loss, accuracy_d = model_inceptionres.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

289/289 - 76s - loss: 0.0457 - accuracy: 0.9855 - 76s/epoch - 265ms/step
 model, accuracy: 98.55%


In [28]:
# model_inceptionres.save('D:/Grad_Proj/colon_histo/main/saved_model/my_model')

INFO:tensorflow:Assets written to: D:/Grad_Proj/colon_histo/main/saved_model/my_model\assets


INFO:tensorflow:Assets written to: D:/Grad_Proj/colon_histo/main/saved_model/my_model\assets


In [29]:
# model_inceptionres.save('D:/Grad_Proj/colon_histo/main/saved_model/my_model_5.h5', save_format='h5')

In [8]:

path = 'D:/Grad_Proj/colon_histo/main/saved_model/my_model_5.h5'
model_inceptionres = tf.keras.models.load_model((path), custom_objects={'KerasLayer':hub.KerasLayer})

In [9]:
model = tf.keras.models.load_model('D:/Grad_Proj/colon_histo/main/saved_model/my_model')


In [16]:
loss, accuracy_d = model.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

4612/4612 - 568s - loss: 0.0457 - accuracy: 0.9855 - 568s/epoch - 123ms/step
 model, accuracy: 98.55%


## Re-Training & Testing Inception_resnet_v2 on 2 classes

In [24]:
batch_size = 16
IMG_SHAPE = 299
base_dir = 'D:/Grad_Proj/colon_histo/sub'
val_base_dir = 'D:/Grad_Proj/colon_histo/sub'

In [25]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(val_base_dir, 'val')
test_dir = os.path.join(val_base_dir, 'test')

In [26]:
training_data = tf.keras.utils.image_dataset_from_directory(directory = train_dir,
                                                           labels = 'inferred',
                                                           label_mode='binary',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           shuffle=True,
                                                           color_mode='rgb')

Found 29773 files belonging to 2 classes.


In [27]:
val_data = tf.keras.utils.image_dataset_from_directory(directory = val_dir,
                                                           labels = 'inferred',
                                                           label_mode='binary',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 3307 files belonging to 2 classes.


In [28]:
test_data = tf.keras.utils.image_dataset_from_directory(directory = test_dir,
                                                           labels = 'inferred',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 1974 files belonging to 2 classes.


In [29]:
@tf.function
def map_fn(img, label):
    image_height = IMG_SHAPE
    image_width = IMG_SHAPE
    img = tf.image.resize(img, [image_height, image_width])
    img /= 255
    return img, label
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

In [30]:
triann = training_data.map(map_fn)
vall = val_data.map(map_fn)
testt = test_data.map(map_fn)

In [23]:
IMAGE_RES = 299

URL = "https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)
with tf.device(device_name=device):
    
    model_inceptionres = tf.keras.Sequential([
      feature_extractor,
        tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(2)
    ])

    model_inceptionres.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              54336736  
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 55,124,706
Trainable params: 787,970
Non-trainable params: 54,336,736
_________________________________________________________________


In [57]:
with tf.device(device_name=device):
    
    model_inceptionres.compile(
      optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001,
      momentum=0.9),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

In [58]:
checkpoint_path = "D:/Grad_Proj/colon_histo/sub/cpk_inception_res/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [59]:
EPOCHS = 2
with tf.device(device_name=device):
    
    history = model_inceptionres.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall,
                        callbacks=[cp_callback])

Epoch 1/2
1861/1861 [==============================] - ETA: 0s - loss: 0.0139 - accuracy: 0.9960
Epoch 1: saving model to D:/Grad_Proj/colon_histo/sub/cpk_inception_res\cp.ckpt
1861/1861 [==============================] - 648s 346ms/step - loss: 0.0139 - accuracy: 0.9960 - val_loss: 0.0258 - val_accuracy: 0.9930
Epoch 2/2
1861/1861 [==============================] - ETA: 0s - loss: 0.0130 - accuracy: 0.9962
Epoch 2: saving model to D:/Grad_Proj/colon_histo/sub/cpk_inception_res\cp.ckpt
1861/1861 [==============================] - 662s 355ms/step - loss: 0.0130 - accuracy: 0.9962 - val_loss: 0.0263 - val_accuracy: 0.9930


In [60]:
loss, accuracy_d = model_inceptionres.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

124/124 - 39s - loss: 0.0994 - accuracy: 0.9681 - 39s/epoch - 313ms/step
 model, accuracy: 96.81%


### Augmenting

In [20]:
# from keras.preprocessing.image import ImageDataGenerator
# from skimage import io
# datagen = ImageDataGenerator(        
#         rotation_range = 40,
#         shear_range = 0.2,
#         zoom_range = 0.2,
#         horizontal_flip = True,
#         brightness_range = (0.5, 1.5))
# import numpy as np
# import os
# from PIL import Image
# image_directory = r'C:/Users/dell g3/Downloads/colon_histo/sub/to_augment/adenocarcinoma/'
# SIZE = 299
# dataset = []
# my_images = os.listdir(image_directory)
# for i, image_name in enumerate(my_images):    
#     if (image_name.split('.')[1] == 'jpg'):        
#         image = io.imread(image_directory + image_name)        
#         image = Image.fromarray(image, 'RGB')        
#         image = image.resize((SIZE,SIZE)) 
#         dataset.append(np.array(image))
# x = np.array(dataset)
# i = 0
# for batch in datagen.flow(x, batch_size=16,
#                           save_to_dir= r'C:/Users/dell g3/Downloads/colon_histo/sub/to_augment/aug_adeno',
#                           save_prefix='dr',
#                           save_format='jpg'):    
#     i += 1    
#     if i > 50:        
#         break

# Re-train entire model 

In [2]:
batch_size = 16
IMG_SHAPE = 299
base_dir = 'D:/Grad_Proj/colon_histo/main'
val_base_dir = 'D:/Grad_Proj/colon_histo/main'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(val_base_dir, 'val')
test_dir = os.path.join(val_base_dir, 'test')



In [3]:
training_data = tf.keras.utils.image_dataset_from_directory(directory = train_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           shuffle=True,
                                                           color_mode='rgb')

val_data = tf.keras.utils.image_dataset_from_directory(directory = val_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

test_data = tf.keras.utils.image_dataset_from_directory(directory = test_dir,
                                                           labels = 'inferred',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 48236 files belonging to 3 classes.
Found 8582 files belonging to 3 classes.
Found 4612 files belonging to 3 classes.


In [4]:
@tf.function
def map_fn(img, label):
    image_height = IMG_SHAPE
    image_width = IMG_SHAPE
    img = tf.image.resize(img, [image_height, image_width])
    img /= 255
    return img, label
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

triann = training_data.map(map_fn)
vall = val_data.map(map_fn)
testt = test_data.map(map_fn)

## Inception_v3

In [5]:
IMAGE_RES = 299

URL = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)
with tf.device(device_name=device):
    
    model_inception3 = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.Dense(3)
    ])

    model_inception3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              21802784  
                                                                 
 dense (Dense)               (None, 3)                 6147      
                                                                 
Total params: 21,808,931
Trainable params: 6,147
Non-trainable params: 21,802,784
_________________________________________________________________


In [6]:
with tf.device(device_name=device):
    
    model_inception3.compile(
      optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

In [7]:
# checkpoint_path = "D:/Grad_Proj/colon_histo/main/cpk_inception_3/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)


# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

In [8]:
EPOCHS = 20
with tf.device(device_name=device):
    
    history = model_inception3.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall)

Epoch 1/20
3015/3015 [==============================] - 450s 144ms/step - loss: 0.1697 - accuracy: 0.9532 - val_loss: 0.0537 - val_accuracy: 0.9860
Epoch 2/20
3015/3015 [==============================] - 548s 182ms/step - loss: 0.0634 - accuracy: 0.9811 - val_loss: 0.0387 - val_accuracy: 0.9890
Epoch 3/20
3015/3015 [==============================] - 596s 198ms/step - loss: 0.0498 - accuracy: 0.9843 - val_loss: 0.0333 - val_accuracy: 0.9890
Epoch 4/20
3015/3015 [==============================] - 631s 209ms/step - loss: 0.0432 - accuracy: 0.9861 - val_loss: 0.0319 - val_accuracy: 0.9903
Epoch 5/20
3015/3015 [==============================] - 624s 207ms/step - loss: 0.0392 - accuracy: 0.9870 - val_loss: 0.0287 - val_accuracy: 0.9911
Epoch 6/20
3015/3015 [==============================] - 639s 212ms/step - loss: 0.0361 - accuracy: 0.9880 - val_loss: 0.0274 - val_accuracy: 0.9917
Epoch 7/20
3015/3015 [==============================] - 639s 212ms/step - loss: 0.0339 - accuracy: 0.9885 - val_

In [9]:
loss, accuracy_d = model_inception3.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

289/289 - 32s - loss: 0.0363 - accuracy: 0.9868 - 32s/epoch - 112ms/step
 model, accuracy: 98.68%


In [ ]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy of Ince")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("D:/Grad_Proj/colon_histo/main/mygraph2.png")

# retraining inception v3

In [29]:
model = tf.keras.models.load_model('D:/Grad_Proj/colon_histo/main/saved_model/my_model_inception3')

In [30]:
EPOCHS = 5
with tf.device(device_name=device):
    
    history = model.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall)

Epoch 1/5
3015/3015 [==============================] - 428s 139ms/step - loss: 0.0225 - accuracy: 0.9926 - val_loss: 0.0216 - val_accuracy: 0.9924
Epoch 2/5
3015/3015 [==============================] - 455s 151ms/step - loss: 0.0221 - accuracy: 0.9927 - val_loss: 0.0215 - val_accuracy: 0.9924
Epoch 3/5
3015/3015 [==============================] - 460s 153ms/step - loss: 0.0217 - accuracy: 0.9928 - val_loss: 0.0217 - val_accuracy: 0.9922
Epoch 4/5
3015/3015 [==============================] - 483s 160ms/step - loss: 0.0213 - accuracy: 0.9929 - val_loss: 0.0216 - val_accuracy: 0.9924
Epoch 5/5
3015/3015 [==============================] - 505s 167ms/step - loss: 0.0210 - accuracy: 0.9933 - val_loss: 0.0214 - val_accuracy: 0.9927


In [31]:
loss, accuracy_d = model.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

289/289 - 36s - loss: 0.0348 - accuracy: 0.9868 - 36s/epoch - 123ms/step
 model, accuracy: 98.68%


In [25]:
# model_inception3.save('D:/Grad_Proj/colon_histo/main/saved_model/my_model_inception3')

INFO:tensorflow:Assets written to: D:/Grad_Proj/colon_histo/main/saved_model/my_model_inception3\assets


INFO:tensorflow:Assets written to: D:/Grad_Proj/colon_histo/main/saved_model/my_model_inception3\assets


In [30]:
# model_inception3.save('D:/Grad_Proj/colon_histo/main/saved_model/my_model_inception3.h5', save_format='h5')

### inception_v1

In [2]:
batch_size = 16
IMG_SHAPE = 224
base_dir = 'D:/Grad_Proj/colon_histo/main'
val_base_dir = 'D:/Grad_Proj/colon_histo/main'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(val_base_dir, 'val')
test_dir = os.path.join(val_base_dir, 'test')


In [3]:
training_data = tf.keras.utils.image_dataset_from_directory(directory = train_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           shuffle=True,
                                                           color_mode='rgb')

val_data = tf.keras.utils.image_dataset_from_directory(directory = val_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

test_data = tf.keras.utils.image_dataset_from_directory(directory = test_dir,
                                                           labels = 'inferred',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 48236 files belonging to 3 classes.
Found 8582 files belonging to 3 classes.
Found 4612 files belonging to 3 classes.


In [4]:
@tf.function
def map_fn(img, label):
    image_height = IMG_SHAPE
    image_width = IMG_SHAPE
    img = tf.image.resize(img, [image_height, image_width])
    img /= 255
    return img, label
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

triann = training_data.map(map_fn)
vall = val_data.map(map_fn)
testt = test_data.map(map_fn)

In [34]:
IMAGE_RES = 224

URL = "https://tfhub.dev/google/imagenet/inception_v1/feature_vector/5"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)
with tf.device(device_name=device):
    
    model_inception1 = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.Dense(3)
    ])

    model_inception1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1024)              5607184   
                                                                 
 dense_1 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 5,610,259
Trainable params: 3,075
Non-trainable params: 5,607,184
_________________________________________________________________


In [35]:
with tf.device(device_name=device):
    
    model_inception1.compile(
      optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001,
      momentum=0.9),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])


In [36]:
EPOCHS = 20
with tf.device(device_name=device):
    
    history = model_inception1.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall)

Epoch 1/20
3015/3015 [==============================] - 407s 132ms/step - loss: 0.1098 - accuracy: 0.9684 - val_loss: 0.0471 - val_accuracy: 0.9878
Epoch 2/20
3015/3015 [==============================] - 424s 140ms/step - loss: 0.0603 - accuracy: 0.9800 - val_loss: 0.0390 - val_accuracy: 0.9890
Epoch 3/20
3015/3015 [==============================] - 233s 77ms/step - loss: 0.0519 - accuracy: 0.9826 - val_loss: 0.0356 - val_accuracy: 0.9901
Epoch 4/20
3015/3015 [==============================] - 177s 58ms/step - loss: 0.0476 - accuracy: 0.9843 - val_loss: 0.0337 - val_accuracy: 0.9904
Epoch 5/20
3015/3015 [==============================] - 179s 59ms/step - loss: 0.0447 - accuracy: 0.9849 - val_loss: 0.0322 - val_accuracy: 0.9907
Epoch 6/20
3015/3015 [==============================] - 189s 62ms/step - loss: 0.0423 - accuracy: 0.9857 - val_loss: 0.0311 - val_accuracy: 0.9910
Epoch 7/20
3015/3015 [==============================] - 192s 63ms/step - loss: 0.0406 - accuracy: 0.9863 - val_loss:

In [37]:
loss, accuracy_d = model_inception1.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

289/289 - 20s - loss: 0.0577 - accuracy: 0.9779 - 20s/epoch - 70ms/step
 model, accuracy: 97.79%


## Inception_v2

In [38]:
IMAGE_RES = 224

URL = "https://tfhub.dev/google/imagenet/inception_v2/feature_vector/5"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)
with tf.device(device_name=device):
    
    model_inception2 = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.Dense(3)
    ])

    model_inception2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 1024)              10173112  
                                                                 
 dense_2 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 10,176,187
Trainable params: 3,075
Non-trainable params: 10,173,112
_________________________________________________________________


In [39]:
with tf.device(device_name=device):
    
    model_inception2.compile(
     optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001,
      momentum=0.9),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

In [40]:
EPOCHS = 20
with tf.device(device_name=device):
    
    history = model_inception2.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall)

Epoch 1/20
3015/3015 [==============================] - 245s 80ms/step - loss: 0.1092 - accuracy: 0.9654 - val_loss: 0.0491 - val_accuracy: 0.9851
Epoch 2/20
3015/3015 [==============================] - 239s 79ms/step - loss: 0.0630 - accuracy: 0.9791 - val_loss: 0.0414 - val_accuracy: 0.9876
Epoch 3/20
3015/3015 [==============================] - 241s 80ms/step - loss: 0.0554 - accuracy: 0.9816 - val_loss: 0.0377 - val_accuracy: 0.9882
Epoch 4/20
3015/3015 [==============================] - 240s 80ms/step - loss: 0.0508 - accuracy: 0.9828 - val_loss: 0.0358 - val_accuracy: 0.9892
Epoch 5/20
3015/3015 [==============================] - 240s 79ms/step - loss: 0.0477 - accuracy: 0.9840 - val_loss: 0.0338 - val_accuracy: 0.9895
Epoch 6/20
3015/3015 [==============================] - 244s 81ms/step - loss: 0.0453 - accuracy: 0.9847 - val_loss: 0.0326 - val_accuracy: 0.9903
Epoch 7/20
3015/3015 [==============================] - 242s 80ms/step - loss: 0.0437 - accuracy: 0.9853 - val_loss: 0

In [41]:
loss, accuracy_d = model_inception2.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

289/289 - 17s - loss: 0.0486 - accuracy: 0.9837 - 17s/epoch - 58ms/step
 model, accuracy: 98.37%


## mobilenet_v3_large

In [81]:
# IMAGE_RES = 224

# URL = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5"
# feature_extractor = hub.KerasLayer(URL,
#   input_shape=(IMAGE_RES, IMAGE_RES, 3),
#   trainable=False)
# with tf.device(device_name=device):
    
#     mobilenet_v3 = tf.keras.Sequential([
#       feature_extractor,
#       tf.keras.layers.Dense(2)
#     ])

#     mobilenet_v3.summary()

In [82]:
# with tf.device(device_name=device):
    
#     mobilenet_v3.compile(
#       optimizer=tf.keras.optimizers.SGD(
#     learning_rate=0.001,
#       momentum=0.9),
#       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#       metrics=['accuracy'])

In [83]:
# EPOCHS = 20
# with tf.device(device_name=device):
    
#     history = mobilenet_v3.fit(triann,
#                         epochs=EPOCHS,
#                         validation_data=vall)

In [84]:
# loss, accuracy_d = mobilenet_v3.evaluate(testt, verbose=2)
# print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

## mobilenet_v2_

In [85]:
# IMAGE_RES = 224

# URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/5"
# feature_extractor = hub.KerasLayer(URL,
#   input_shape=(IMAGE_RES, IMAGE_RES, 3),
#   trainable=False)
# with tf.device(device_name=device):
    
#     mobilenet_v2 = tf.keras.Sequential([
#       feature_extractor,
#       tf.keras.layers.Dense(2)
#     ])

#     mobilenet_v2.summary()

In [86]:
# with tf.device(device_name=device):
    
#     mobilenet_v2.compile(
#       optimizer=tf.keras.optimizers.SGD(
#     learning_rate=0.001,
#       momentum=0.9),
#       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#       metrics=['accuracy'])

In [87]:
# EPOCHS = 20
# with tf.device(device_name=device):
    
#     history = mobilenet_v2.fit(triann,
#                         epochs=EPOCHS,
#                         validation_data=vall)

In [88]:
# loss, accuracy_d = mobilenet_v2.evaluate(testt, verbose=2)
# print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

## resnet_v1_101

In [42]:
IMAGE_RES = 224

URL = "https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/5"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)
with tf.device(device_name=device):
    
    
    resnet_v1 = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.Dense(3)
    ])

    resnet_v1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_5 (KerasLayer)  (None, 2048)              42605504  
                                                                 
 dense_3 (Dense)             (None, 3)                 6147      
                                                                 
Total params: 42,611,651
Trainable params: 6,147
Non-trainable params: 42,605,504
_________________________________________________________________


In [43]:
with tf.device(device_name=device):
    
    resnet_v1.compile(
      optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001,
      momentum=0.9),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

In [44]:
EPOCHS = 20
with tf.device(device_name=device):
    
    history = resnet_v1.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall)

Epoch 1/20
3015/3015 [==============================] - 472s 154ms/step - loss: 0.0647 - accuracy: 0.9767 - val_loss: 0.0303 - val_accuracy: 0.9902
Epoch 2/20
3015/3015 [==============================] - 461s 153ms/step - loss: 0.0388 - accuracy: 0.9867 - val_loss: 0.0255 - val_accuracy: 0.9911
Epoch 3/20
3015/3015 [==============================] - 461s 153ms/step - loss: 0.0335 - accuracy: 0.9887 - val_loss: 0.0233 - val_accuracy: 0.9918
Epoch 4/20
3015/3015 [==============================] - 463s 153ms/step - loss: 0.0300 - accuracy: 0.9894 - val_loss: 0.0222 - val_accuracy: 0.9924
Epoch 5/20
3015/3015 [==============================] - 466s 154ms/step - loss: 0.0275 - accuracy: 0.9908 - val_loss: 0.0213 - val_accuracy: 0.9929
Epoch 6/20
3015/3015 [==============================] - 469s 155ms/step - loss: 0.0262 - accuracy: 0.9914 - val_loss: 0.0208 - val_accuracy: 0.9931
Epoch 7/20
3015/3015 [==============================] - 474s 157ms/step - loss: 0.0247 - accuracy: 0.9918 - val_

In [45]:
loss, accuracy_d = resnet_v1.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

289/289 - 38s - loss: 0.0501 - accuracy: 0.9831 - 38s/epoch - 130ms/step
 model, accuracy: 98.31%


## resnet_v2_101

In [8]:
IMAGE_RES = 224

URL = "https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)
with tf.device(device_name=device):
    
    
    resnet_v2 = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.Dense(3)
    ])

    resnet_v2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 2048)              42626560  
                                                                 
 dense_1 (Dense)             (None, 3)                 6147      
                                                                 
Total params: 42,632,707
Trainable params: 6,147
Non-trainable params: 42,626,560
_________________________________________________________________


In [9]:
with tf.device(device_name=device):
    
    resnet_v2.compile(
      optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001,
      momentum=0.9),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

In [10]:
EPOCHS = 20
with tf.device(device_name=device):
    
    history = resnet_v2.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall)

Epoch 1/20
3015/3015 [==============================] - 1040s 341ms/step - loss: 0.0711 - accuracy: 0.9749 - val_loss: 0.0369 - val_accuracy: 0.9887
Epoch 2/20
3015/3015 [==============================] - 573s 190ms/step - loss: 0.0443 - accuracy: 0.9846 - val_loss: 0.0319 - val_accuracy: 0.9897
Epoch 3/20
3015/3015 [==============================] - 579s 192ms/step - loss: 0.0384 - accuracy: 0.9866 - val_loss: 0.0280 - val_accuracy: 0.9911
Epoch 4/20
3015/3015 [==============================] - 590s 195ms/step - loss: 0.0349 - accuracy: 0.9881 - val_loss: 0.0269 - val_accuracy: 0.9918
Epoch 5/20
3015/3015 [==============================] - 580s 192ms/step - loss: 0.0329 - accuracy: 0.9890 - val_loss: 0.0283 - val_accuracy: 0.9913
Epoch 6/20
3015/3015 [==============================] - 581s 193ms/step - loss: 0.0310 - accuracy: 0.9892 - val_loss: 0.0265 - val_accuracy: 0.9918
Epoch 7/20
3015/3015 [==============================] - 588s 195ms/step - loss: 0.0296 - accuracy: 0.9898 - val

In [11]:
loss, accuracy_d = resnet_v2.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

289/289 - 49s - loss: 0.0606 - accuracy: 0.9820 - 49s/epoch - 168ms/step
 model, accuracy: 98.20%


## efficientnet_v2_imagenet21k_m

In [97]:
# batch_size = 4
# IMG_SHAPE = 480
# base_dir = 'C:/Users/dell g3/Downloads/colon_histo/sub'
# val_base_dir = 'C:/Users/dell g3/Downloads/colon_histo/sub'

# train_dir = os.path.join(base_dir, 'train')
# val_dir = os.path.join(val_base_dir, 'val')
# test_dir = os.path.join(val_base_dir, 'test')

In [98]:
# training_data = tf.keras.utils.image_dataset_from_directory(directory = train_dir,
#                                                            labels = 'inferred',
#                                                            label_mode='binary',
#                                                            batch_size=batch_size,
#                                                            image_size=(IMG_SHAPE, IMG_SHAPE),
#                                                            shuffle=True,
#                                                            color_mode='rgb')

# val_data = tf.keras.utils.image_dataset_from_directory(directory = val_dir,
#                                                            labels = 'inferred',
#                                                            label_mode='binary',
#                                                            batch_size=batch_size,
#                                                            image_size=(IMG_SHAPE, IMG_SHAPE),
#                                                            color_mode='rgb')

# test_data = tf.keras.utils.image_dataset_from_directory(directory = test_dir,
#                                                            labels = 'inferred',
#                                                            batch_size=batch_size,
#                                                            image_size=(IMG_SHAPE, IMG_SHAPE),
#                                                            color_mode='rgb')

In [99]:
# @tf.function
# def map_fn(img, label):
#     image_height = IMG_SHAPE
#     image_width = IMG_SHAPE
#     img = tf.image.resize(img, [image_height, image_width])
#     img /= 255
#     return img, label
# device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

# triann = training_data.map(map_fn)
# vall = val_data.map(map_fn)
# testt = test_data.map(map_fn)

In [100]:
# IMAGE_RES = 480

# URL = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2"
# feature_extractor = hub.KerasLayer(URL,
#   input_shape=(IMAGE_RES, IMAGE_RES, 3),
#   trainable=False)
# with tf.device(device_name=device):
    
    
#     efficientnet_v2_m = tf.keras.Sequential([
#       feature_extractor,
#       tf.keras.layers.Dense(2)
#     ])

#     efficientnet_v2_m.summary()

In [101]:
# with tf.device(device_name=device):
    
#     efficientnet_v2_m.compile(
#       optimizer=tf.keras.optimizers.SGD(
#     learning_rate=0.001,
#       momentum=0.9),
#       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#       metrics=['accuracy'])

In [102]:
# EPOCHS = 20
# with tf.device(device_name=device):
    
#     history = efficientnet_v2_m.fit(triann,
#                         epochs=EPOCHS,
#                         validation_data=vall)

In [103]:
# loss, accuracy_d = efficientnet_v2_m.evaluate(testt, verbose=2)
# print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

## efficientnet_v2_imagenet21k_l

In [104]:
# IMAGE_RES = 480

# URL = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/feature_vector/2"
# feature_extractor = hub.KerasLayer(URL,
#   input_shape=(IMAGE_RES, IMAGE_RES, 3),
#   trainable=False)
# with tf.device(device_name=device):
    
    
#     efficientnet_v2_l = tf.keras.Sequential([
#       feature_extractor,
#       tf.keras.layers.Dense(2)
#     ])

#     efficientnet_v2_l.summary()

In [105]:
# with tf.device(device_name=device):
    
#     efficientnet_v2_l.compile(
#       optimizer=tf.keras.optimizers.SGD(
#     learning_rate=0.001,
#       momentum=0.9),
#       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#       metrics=['accuracy'])

In [106]:
# EPOCHS = 20
# with tf.device(device_name=device):
    
#     history = efficientnet_v2_l.fit(triann,
#                         epochs=EPOCHS,
#                         validation_data=vall)

In [107]:
# loss, accuracy_d = efficientnet_v2_l.evaluate(testt, verbose=2)
# print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

## efficientnet_v2_imagenet21k_s

In [12]:
batch_size = 4
IMG_SHAPE = 384
base_dir = 'D:/Grad_Proj/colon_histo/main'
val_base_dir = 'D:/Grad_Proj/colon_histo/main'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(val_base_dir, 'val')
test_dir = os.path.join(val_base_dir, 'test')

In [14]:
training_data = tf.keras.utils.image_dataset_from_directory(directory = train_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           shuffle=True,
                                                           color_mode='rgb')

val_data = tf.keras.utils.image_dataset_from_directory(directory = val_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

test_data = tf.keras.utils.image_dataset_from_directory(directory = test_dir,
                                                           labels = 'inferred',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 48236 files belonging to 3 classes.
Found 8582 files belonging to 3 classes.
Found 4612 files belonging to 3 classes.


In [15]:
@tf.function
def map_fn(img, label):
    image_height = IMG_SHAPE
    image_width = IMG_SHAPE
    img = tf.image.resize(img, [image_height, image_width])
    img /= 255
    return img, label
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

triann = training_data.map(map_fn)
vall = val_data.map(map_fn)
testt = test_data.map(map_fn)

In [16]:
IMAGE_RES = 384

URL = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)
with tf.device(device_name=device):
    
    
    efficientnet_v2_s = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.Dense(3)
    ])

    efficientnet_v2_s.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              20331360  
                                                                 
 dense_2 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 20,335,203
Trainable params: 3,843
Non-trainable params: 20,331,360
_________________________________________________________________


In [17]:
with tf.device(device_name=device):
    
    efficientnet_v2_s.compile(
      optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001,
      momentum=0.9),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

In [24]:
EPOCHS = 10
with tf.device(device_name=device):
    
    history = efficientnet_v2_s.fit(triann,
                        epochs=EPOCHS,
                        validation_data=vall)

Epoch 1/10
12059/12059 [==============================] - 1370s 113ms/step - loss: 0.0112 - accuracy: 0.9964 - val_loss: 0.0120 - val_accuracy: 0.9956
Epoch 2/10
12059/12059 [==============================] - 1425s 118ms/step - loss: 0.0110 - accuracy: 0.9965 - val_loss: 0.0121 - val_accuracy: 0.9958
Epoch 3/10
12059/12059 [==============================] - 1613s 134ms/step - loss: 0.0108 - accuracy: 0.9962 - val_loss: 0.0125 - val_accuracy: 0.9958
Epoch 4/10
12059/12059 [==============================] - 1421s 118ms/step - loss: 0.0104 - accuracy: 0.9963 - val_loss: 0.0121 - val_accuracy: 0.9963
Epoch 5/10
12059/12059 [==============================] - 1371s 114ms/step - loss: 0.0100 - accuracy: 0.9965 - val_loss: 0.0121 - val_accuracy: 0.9963
Epoch 6/10
 5137/12059 [===========>..................] - ETA: 11:41 - loss: 0.0081 - accuracy: 0.9973

KeyboardInterrupt: 

In [25]:
loss, accuracy_d = efficientnet_v2_s.evaluate(testt, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * accuracy_d))

1153/1153 - 101s - loss: 0.0869 - accuracy: 0.9811 - 101s/epoch - 88ms/step
 model, accuracy: 98.11%


## VGG16

In [15]:
batch_size = 16
IMG_SHAPE = 224
base_dir = 'D:/Grad_Proj/colon_histo/main'
val_base_dir = 'D:/Grad_Proj/colon_histo/main'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(val_base_dir, 'val')
test_dir = os.path.join(val_base_dir, 'test')

In [16]:
training_data = tf.keras.utils.image_dataset_from_directory(directory = train_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           shuffle=True,
                                                           color_mode='rgb')

val_data = tf.keras.utils.image_dataset_from_directory(directory = val_dir,
                                                           labels = 'inferred',
                                                           label_mode='int',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

test_data = tf.keras.utils.image_dataset_from_directory(directory = test_dir,
                                                           labels = 'inferred',
                                                           batch_size=batch_size,
                                                           image_size=(IMG_SHAPE, IMG_SHAPE),
                                                           color_mode='rgb')

Found 29773 files belonging to 2 classes.
Found 3307 files belonging to 2 classes.
Found 1974 files belonging to 2 classes.


In [17]:
@tf.function
def map_fn(img, label):
    image_height = IMG_SHAPE
    image_width = IMG_SHAPE
    img = tf.image.resize(img, [image_height, image_width])
    img /= 255
    return img, label
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

triann = training_data.map(map_fn)
vall = val_data.map(map_fn)
testt = test_data.map(map_fn)

In [18]:
import os
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

In [19]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):

    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)

    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    model = Model(inputs=conv_base.input, outputs=output_layer)

    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [22]:
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.0001)
n_classes=3
n_epochs = 20
BATCH_SIZE = 16

vgg_model_ft = create_model(input_shape, n_classes, optim_1, fine_tune=2)

NameError: name 'Adam' is not defined

In [34]:
tl_checkpoint_1 = ModelCheckpoint(filepath='VGG16_weightsbest.hdf5',
                                  save_best_only=True,
                                  verbose=1)

In [35]:
with tf.device(device_name=device):
    vgg16_ft_history = vgg_model_ft.fit(triann,
                                      batch_size=BATCH_SIZE,
                                      epochs=n_epochs,
                                      validation_data=vall,
                                      callbacks=[tl_checkpoint_1],
                                      verbose=1)

Epoch 1/20
1861/1861 [==============================] - ETA: 0s - loss: 0.0055 - accuracy: 0.9988
Epoch 1: val_loss improved from inf to 0.04740, saving model to VGG16_weightsbest.hdf5
1861/1861 [==============================] - 310s 166ms/step - loss: 0.0055 - accuracy: 0.9988 - val_loss: 0.0474 - val_accuracy: 0.9867
Epoch 2/20
  21/1861 [..............................] - ETA: 4:34 - loss: 1.9767e-04 - accuracy: 1.0000

KeyboardInterrupt: 

In [36]:
vgg_model_ft.load_weights('VGG16_weightsbest.hdf5')

In [38]:
train_gen = ImageDataGenerator(preprocessing_function= preprocess_input)
traingen = train_gen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               class_mode='binary',
                                               batch_size=16, 
                                               shuffle=True,
                                               seed=42)

Found 29773 images belonging to 2 classes.


In [39]:
test_gen = ImageDataGenerator(preprocessing_function= preprocess_input)
testgen = test_gen.flow_from_directory(test_dir,
                                      target_size=(224, 224),
                                      batch_size=1,
                                      shuffle=False,
                                      seed=42)

Found 1974 images belonging to 2 classes.


In [40]:
true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds = vgg_model_ft.predict(testgen)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

1974/1974 [==============================] - 33s 16ms/step


In [41]:
from sklearn.metrics import accuracy_score

vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))

VGG16 Model Accuracy without Fine-Tuning: 74.01%


## VGG19

In [42]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):

    conv_base = VGG19(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)

    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    model = Model(inputs=conv_base.input, outputs=output_layer)

    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [50]:
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.0001)
n_classes=2
n_epochs = 20
BATCH_SIZE = 4

vgg19_model_ft = create_model(input_shape, n_classes, optim_1, fine_tune=2)

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [ ]:
tl_checkpoint_1 = ModelCheckpoint(filepath='VGG19_weightsbest.hdf5',
                                  save_best_only=True,
                                  verbose=1)

In [ ]:
with tf.device(device_name=device):
    vgg19_ft_history = vgg19_model_ft.fit(triann,
                                      batch_size=BATCH_SIZE,
                                      epochs=n_epochs,
                                      validation_data=vall,
                                      callbacks=[tl_checkpoint_1],
                                      verbose=1)

In [ ]:
vgg_model_ft.load_weights('VGG19_weightsbest.hdf5')